![](https://drive.google.com/uc?export=view&id=1-5X9OUkA-C2Ih1gOS9Jd7GmkTWUEpDg1)

Asignatura: Introducción a Data Science

Profesor: Juan Bekios Calfa

---



# Laboratorio 03: Teoría de grafos

En los cursos de ciencias básicas de computación se estudia con frecuencia el problema del vendedor viajero (problema del vendedor ambulante, problema del agente viajero o problema del viajante (**TSP** por sus siglas en inglés, *Travelling Salesman Problem*) y se definen como: dada una lista de ciudades y las distancias entre cada par de ellas, ¿cuál es la ruta más corta posible que visita cada ciudad exactamente una vez y al finalizar regresa a la ciudad origen? Este es un problema NP-Hard dentro en la optimización combinatoria, muy importante en investigación operativa y en ciencias de la computación. [wikipedia](https://es.wikipedia.org/wiki/Problema_del_viajante)

También, existe otro tipo de problemas interesantes y similares al del vendedor viajero. Uno de ellos, es el que implementaremos en este laboratorio, denominado problema del cartero chino (PCC). El problema consiste en **encontrar el camino más corto o circuito cerrado**, que **visite cada arista de un grafo (conectado) no direccionado**, o sea, que pase al menos una vez por cada arista del grafo, volviendo al punto (o nodo) de partida. Cuando el grafo posee un circuito euleriano (un paseo cerrado que alcance toda arista solamente una vez), ese circuito es una solución óptima. [wikipedia](https://es.wikipedia.org/wiki/Problema_del_cartero_chino)

**Problema**:

El parque Sleeping Giant State, ubicado en  New Haven, Connecticut, Estados Unidos. Es un parque compuesto de un sin número de senderos que permiten a los turistas caminar por ellos. Como el parque tiene una extensión de terreno muy grande, mucha gente decide recorrer todos los senderos en un solo día. 

![](https://www.wnpr.org/sites/wnpr/files/styles/x_large/public/201906/sleeping__3_of_5_.jpg)

Como el calcular la ruta adecuada no es fácil recurriremos al material visto en clases para poder encontrar la ruta óptima que nos permita recorrer todos los senderos en un mínimo de tiempo.

Para poder resolver este problema contamos con un plano que indica cada una de esas rutas.

![](https://giantstout.files.wordpress.com/2010/10/sg-trail-map.jpg?w=640&h=460)

El problema está basado en [link](https://www.datacamp.com/community/tutorials/networkx-python-graph-tutorial).

## Librerías que se utilzarán en el laboratorio




In [ ]:
import networkx as nx # Librería para grafos
import pandas as pd # Librería para manejar datos (dataframe)
import matplotlib.pyplot as plt # Librería para gráficos

## 1. Carga de datos

Para resolver este problema cargaremos un archivo que contiene una lista de con los arcos del grafo y cada fila contiene los siguientes campos:

*   **node1** y **node2**: Nombre de los nodos conectados.
*   **trail**: Atributo que indica un atributo del arco. (Por ejemplo, rs=cuadro rojo)
*   **distance**: Atributo del arco que indica la longitud del sendero en millas.
*   **color**: Color utilizado para indicar el sendero trazado.
*   **estimate**: Distancia del arco estimada a partir de la observación de los senderos del mapa (1=SI, 0=N=), ya que algunas distancias no se proporcionan. Esto es solo de referencia y no será utilizado para el análisis.





In [ ]:
# Grab edge list data hosted on Gist
lista_arcos = pd.read_csv('https://gist.githubusercontent.com/brooksandrew/e570c38bcc72a8d102422f2af836513b/raw/89c76b2563dbc0e88384719a35cba0dfc04cd522/edgelist_sleeping_giant.csv')

lista_arcos.head(10)

### Lista de nodos

Las listas de nodos suelen ser opcionales en `networkx`, y otras bibliotecas de gráficos, cuando se proporcionan listas de arcos ya vienen los nombres por lo que no es necesario identificarlos. Sin embargo, en este caso, hay algunos atributos de nodo que nos gustaría añadir: Coordenadas X, Y de los nodos (intersecciones de senderos) para que pueda trazar su gráfica con el mismo diseño que el mapa de senderos.

*    id: nombre del nodo correspondiente al nodo1 y al nodo2 en la lista de bordes.
*    X: posición horizontal/coordenada del nodo correspondiente al topleft.
*    Y: posición/coordinación vertical del nodo relativo al topleft.

In [ ]:
# Archivo de la lista de nodos
lista_nodos = pd.read_csv('https://gist.githubusercontent.com/brooksandrew/f989e10af17fb4c85b11409fea47895b/raw/a3a8da0fa5b094f1ca9d82e1642b384889ae16e8/nodelist_sleeping_giant.csv')

lista_nodos.head(10)

## 2. Creando el grafo

Usaremos la lista de arcos (aristas) y la lista de nodos para crear el grafo utilizando las librerías de `networkx`.

In [ ]:
# Crear un grafo limpio

g = nx.Graph()

Construimos el grafo ingresando cada uno de sus arcos y sus correspondientes atributos al grafo `g`.

In [ ]:
# Agregar aristas y atributos a las aristas (arcos)

for index, datos_arco in lista_arcos.iterrows():
  print(datos_arco)
  #print(f'{datos_arco["node1"]}')
  #print(f'Indice {index}:')
  #print(datos_arco)
  #print('-'*40)
  #print(datos_arco[2:].to_dict())
  #print('='*40)
  #g.add_edge(datos_arco[0], datos_arco[1], attr_dict=datos_arco[2:].to_dict())

Para ilustrar los valores devueltos revisamos la última fila examinada en el ciclo `for`.

In [ ]:
# Ejemplo de lista de arcos
print(datos_arco[0]) # nodo 1
print(datos_arco[1]) # nodo 2
print(datos_arco[2:].to_dict()) # atributos asociados al arco

De la misma forma que se agregaron atributos a los arcos del grafo. Agregaremos los atributos a cada nodo.

In [ ]:
# Agregar atributos a los nodos
for _, fila_nodo in lista_nodos.iterrows():
  print(fila_nodo['id'], fila_nodo[1:].to_dict())
  #g.nodes[fila_nodo['id']].update(fila_nodo[1:].to_dict())

Si anallizamos la última fila de la lista de nodos podemos ver:

In [ ]:
# Ejemplo de lista de nodos
print(fila_nodo)

## 3. Explorando el grafo creado

El grafo creado almacena la información de cada arco en una tupla de tres elementos. Dos son string y el tercero es un diccionario.

In [ ]:
# Visualizar el grafo
g.edges.data()

# Listar la información de las aristas (arcos)
#for arc in g.edges.data():
#  print(arc)

### Imprimir los primeros cinco arcos

In [ ]:
# Visualizar los primeros cinco arcos del grafo
list(g.edges(data=True))[:5]

También podemos visualizar la información de los nodos:

In [ ]:
# Visualizar los 10 primeros nodos
list(g.nodes(data=True))[0:10]

### Estadísticas del grafo

También se pueden imprimir algunas valores que resumen los elementos del grafo.

In [ ]:
print(f'Número de arcos: {g.number_of_edges()}')
print(f'Número de nodos: {g.number_of_nodes()}')

## 4. Visualización del grafo

Primero hay que manipular las posiciones de los nodos del gráfico en un diccionario. Esto te permitirá recrear el gráfico usando la misma disposición que el mapa de senderos real. **Y** se anula para transformar el origen del eje Y de la parte superior izquierda a la inferior izquierda.

In [ ]:
# Defino las posiciones del nodo de la estructura de datos (dict) para graficarlo
node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in g.nodes(data=True)}

# Previsualización node_positions 
print(dict(list(node_positions.items())[0:5]))
node_new_colors = dict(list(node_positions.items())[0:5])
print(node_new_colors)

**Colores**: Ahora configuraremos los colores de los arcos del gráfico en una simple lista para se pueda visualizar los caminos (senderos) por su color.

In [ ]:
# Definir una estructura de datos (list) para los colores de los arcos en el gráfico
edge_colors = [data['attr_dict']['color'] for _, _, data in g.edges(data=True)]

# Preview first 10
edge_colors[0:10]

## 5. Graficar

Ahora puedes hacer un buen gráfico que se alinee bien con el mapa de senderos del parque Sleeping Giant.

In [ ]:
plt.figure(figsize=(12, 9))
nx.draw(g, pos=node_positions, edge_color=edge_colors, node_size=10, node_color='black')
plt.title('Grafo que representa los senderos de Sleeping Giant', size=15)
plt.xlabel('Posición X')
plt.ylabel('Posición Y')
plt.show()

Se puede visualizar el grafo de diferentes formas dependiendo de los parámetros que le pasemos al método `draw`.

In [ ]:
plt.figure(figsize=(12, 9))
nx.draw(g, pos=node_positions, edge_color=edge_colors, node_size=30, node_color='black', with_labels = True)
plt.title('Grafo que representa los senderos de Sleeping Giant', size=15)
plt.axis('on')
plt.xlabel('Posición X')
plt.ylabel('Posición Y')

# Tarea

Resolver los siguientes ejercicios para el próximo laboratorio:

* Dibujar el grafo con aristas grises y un alfa=0.4. Dibujar de color rojo todos los nodos que tengan un número de senderos de entrada o salida impar.
*  Dibujar el grafo con aristas grises y un alfa=0.4. Dibujar de color azul todos los nodos que tengan un número de senderos de entrada o salida par.
* Dibujar el grafo con aristas grises y un alfa=0.4. Dibujar de color rojo todos los nodos que tengan un número de senderos de entrada o salida impar y de color azul los que tengan nodos par.
* Pintar los nodos de grado menor con colores claros y los nodo de grado mayor con tonos fuertes (use una paleta de gradientes).
* Encontrar el camino más corto entre rc_end_north y rs_end_south. Dibujar la ruta en el grafo. Donde los arcos rojos representen la ruta mínima y los grises las rutas que no son visitadas. Los arcos que son visitados colocar la distacia. 
* ¿Existen nodos que forman comunidades en el grafo?. Si es así, dibujar un grafo donde se muestren las comunidades. Arcos o rutas en rojo representan las comunidades conectadas y las grises nodos no conectados.
* Pinte de color rojo, y aumente su tamaño a 40, dle nodo más importante y el resto de color negro.

In [ ]:
# Código de ejemplo
plt.figure(figsize=(12, 10))
nx.draw_networkx_nodes(g, node_positions, node_size=20, node_color="red")
# Visualizar los 10 primeros nodos
lista_nodos_verdes = [ n[0] for n in list(g.nodes(data=True))[0:10]]
nx.draw_networkx_nodes(g, node_positions, nodelist=lista_nodos_verdes, node_size=60, node_color="green")

nx.draw_networkx_edges(g, node_positions, alpha=0.4)
plt.axis('on')
#plt.grid('on')
plt.title('Cambiando el color de los arcos del grafo')
plt.show()

In [ ]:
# Grado de un nodo
for nodo in g.degree():
  print(nodo)